
# Rule suggestions


This notebook presents some possible rules that could be extacted, paired with their appropriate datasets.

---
### **Rule 1**: (A, "parents", B) and (A, "parents", C) and (B, "gender", "male") --> (C, "gender", "female")
"If some A has different parents B and C, and B is male, then C is female."


**Dataset**: fb13

This rule is somewhat homophobic, but I assume that the embedding will support it.


---
### **Rule 2**: (A, "daughterOf", B) and (B, "motherOf", C) --> (A, "sisterOf", C)
"If some A is the daughter of some B, and that B is a mother of C, then A is a sister of C."



**Dataset**: kinship

---
### **Rule 3**: (A, "wasBornIn", B) and (A, "livesIn", B) --> (A, "isCitizenOf", B)
"If some A was born in B and lives in B, then they are a citizen of B."

**Dataset**: yago3_10

---
### **Rule 4**: (A, "livesIn", B) and (B, "isCapitalOf", C) --> (A, "livesIn", C)
"If some A lives in B, which is the capital of C, then A lives in C."


**Dataset**: yago3_10

---
### **Rule 5**: (A, "isCapitolOf", B) and (C, "isLeaderOf" B) --> (C, "isCitizenOf", B)
"If A is the capitol of B and C is the leader of B, then C is a citizen of B." (You cannot be a leader of a country without being a citizen of that country.)

***Problematic***: B is not nessecarily a country in this dataset. Could have written (B "hasCurrency" A), but then we encounter the same problem.

**Dataset**: yago3_10


---
### **Rule 6**: (A, "playsFor", B) --> (A, "gender", "male")
"If A plays for B, then A is a male."

This is an odd rule which might be supported by the dataset?? Would be interesting to check out.


**Dataset**: yago3_10

In [1]:
import numpy as np
import ampligraph
import tensorflow as tf
from ampligraph.datasets import load_yago3_10
from ampligraph.evaluation import evaluate_performance
from ampligraph.evaluation import train_test_split_no_unseen 
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
from ampligraph.latent_features import save_model
from signature_tools import subset_by_signature, subset_by_frequency, most_frequent_objects, most_frequent_predicates, most_frequent_targets

## Yago3_10

In [2]:
yago = load_yago3_10()
yago = np.concatenate([yago['train'], yago['valid'], yago['test']]) # combine the split data

In [3]:
most_frequent_objects(yago)

array([[264, 'Frankfurt_Airport'],
       [259, 'Amsterdam_Airport_Schiphol'],
       [250, 'Charles_de_Gaulle_Airport'],
       [230, 'Istanbul_Atatürk_Airport'],
       [210, 'United_States'],
       [206, 'Munich_Airport'],
       [202, 'Barcelona–El_Prat_Airport'],
       [200, 'Dubai_International_Airport'],
       [191, 'Brussels_Airport'],
       [179, 'Leonardo_da_Vinci–Fiumicino_Airport']], dtype=object)

In [4]:
most_frequent_predicates(yago, n = 40)

array([[377143, 'isAffiliatedTo'],
       [324048, 'playsFor'],
       [89495, 'isLocatedIn'],
       [66764, 'hasGender'],
       [45410, 'wasBornIn'],
       [32479, 'actedIn'],
       [32338, 'isConnectedTo'],
       [24277, 'hasWonPrize'],
       [10801, 'influences'],
       [9340, 'diedIn'],
       [7827, 'hasMusicalRole'],
       [7432, 'graduatedFrom'],
       [7006, 'created'],
       [6102, 'wroteMusicFor'],
       [5530, 'directed'],
       [5190, 'participatedIn'],
       [5111, 'hasChild'],
       [5099, 'happenedIn'],
       [3795, 'isMarriedTo'],
       [3482, 'isCitizenOf'],
       [3419, 'worksAt'],
       [3114, 'edited'],
       [3011, 'livesIn'],
       [2587, 'hasCapital'],
       [2186, 'isPoliticianOf'],
       [1320, 'dealsWith'],
       [966, 'isLeaderOf'],
       [921, 'hasAcademicAdvisor'],
       [733, 'owns'],
       [558, 'hasNeighbor'],
       [394, 'imports'],
       [381, 'exports'],
       [300, 'hasOfficialLanguage'],
       [243, 'isInterestedIn'],
 

In [5]:
most_frequent_targets(yago)

array([[61599, 'male'],
       [12309, 'United_States'],
       [5165, 'female'],
       [1743, 'London'],
       [1733, 'United_Kingdom'],
       [1673, 'wordnet_guitar_103467517'],
       [1578, 'France'],
       [1443, 'New_York_City'],
       [1384, 'Birmingham_City_F.C.'],
       [1373, 'Los_Angeles']], dtype=object)

### Generate subset

It seems that airports make up a large part of the dataset, and these are related to eachother by the predicate "isConnectedTo". 

So, we will generate a subset containing only triplets with the predicate "isConnectedTo" that include entities that appear in at least 50 triplets.

In [45]:
subset = subset_by_signature(yago, [], ["playsFor"], [])

In [48]:
subset[:100]

array([['Boo_Young-tae', 'playsFor', 'Yangju_Citizen_FC'],
       ['Josh_Smith_(soccer)', 'playsFor', 'Trinity_University_(Texas)'],
       ['George_Mallia', 'playsFor', 'Malta_national_football_team'],
       ['Ariel_Garcé', 'playsFor', 'Rosario_Central'],
       ['Trevor_Senior', 'playsFor', 'Dorchester_Town_F.C.'],
       ['Emanuele_Concetti', 'playsFor', 'U.S._Pergolettese_1932'],
       ['Warren_Bradley_(footballer)', 'playsFor',
        'Macclesfield_Town_F.C.'],
       ['Simone_Zaza', 'playsFor', 'F.C._Esperia_Viareggio'],
       ['Ricardo_Moniz', 'playsFor', 'Helmond_Sport'],
       ['László_Sternberg', 'playsFor', 'New_York_Americans_(soccer)'],
       ['Lee_Clarke', 'playsFor',
        'Northern_Ireland_national_under-21_football_team'],
       ['Nicola_Sansone', 'playsFor', 'Parma_F.C.'],
       ['Edgardo_Abdala', 'playsFor', 'Huachipato'],
       ['Júnior_Maranhão', 'playsFor', 'Club_Sportivo_Sergipe'],
       ['Xavier_Hochstrasser', 'playsFor', 'Calcio_Padova'],
       ['N

In [8]:
entities = np.unique(np.concatenate([subset[:, 0], subset[:, 2]]))
entities

array(['14th_Dalai_Lama', 'A._C._Grayling', 'A._J._John,_Anaparambil',
       ..., 'Łódź', 'Żagań', 'Željko_Ražnatović'], dtype=object)

In [9]:
relations = np.unique(subset[:, 1])
relations

array(['livesIn'], dtype=object)

In [10]:
#subset = subset_by_frequency(subset, 1, 1)

In [11]:
subset.shape

(3011, 3)

# 2. Defining train and test datasets

As is typical in machine learning, we need to split our dataset into training and test (and sometimes validation) datasets. 

What differs from the standard method of randomly sampling N points to make up our test set, is that our data points are two entities linked by some relationship, and we need to take care to ensure that all entities are represented in train and test sets by at least one triple. 

To accomplish this, AmpliGraph provides the [`train_test_split_no_unseen`](https://docs.ampligraph.org/en/latest/generated/ampligraph.evaluation.train_test_split_no_unseen.html#train-test-split-no-unseen) function.  

For sake of example, we will create a small test size that includes only 100 triples:

In [12]:
from ampligraph.evaluation import train_test_split_no_unseen 

X_train, X_test = train_test_split_no_unseen(subset, test_size=100) 

Our data is now split into train/test sets. If we need to further divide into a validation dataset we can just repeat using the same procedure on the test set (and adjusting the split percentages). 

In [13]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (2911, 3)
Test set size:  (100, 3)


In [14]:
["Trondheim_Airport", "hasGender", "Brisbane_Airport"] in subset

False

---
# 3. Training a model 

AmpliGraph has implemented [several Knoweldge Graph Embedding models](https://docs.ampligraph.org/en/latest/ampligraph.latent_features.html#knowledge-graph-embedding-models) (TransE, ComplEx, DistMult, HolE), but to begin with we're just going to use the [ComplEx](https://docs.ampligraph.org/en/latest/generated/ampligraph.latent_features.ComplEx.html#ampligraph.latent_features.ComplEx) model (with  default values), so lets import that:

In [15]:
from ampligraph.latent_features import ComplEx

Lets go through the parameters to understand what's going on:

- **`k`** : the dimensionality of the embedding space
- **`eta`** ($\eta$) : the number of negative, or false triples that must be generated at training runtime for each positive, or true triple
- **`batches_count`** : the number of batches in which the training set is split during the training loop. If you are having into low memory issues than settings this to a higher number may help.
- **`epochs`** : the number of epochs to train the model for.
- **`optimizer`** : the Adam optimizer, with a learning rate of 1e-3 set via the *optimizer_params* kwarg.
- **`loss`** : pairwise loss, with a margin of 0.5 set via the *loss_params* kwarg.
- **`regularizer`** : $L_p$ regularization with $p=2$, i.e. l2 regularization. $\lambda$ = 1e-5, set via the *regularizer_params* kwarg. 

Now we can instantiate the model:


In [16]:
model = ComplEx(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

## Filtering negatives

AmpliGraph aims to follow scikit-learn's ease-of-use design philosophy and simplify everything down to **`fit`**, **`evaluate`**, and **`predict`** functions. 

However, there are some knowledge graph specific steps we must take to ensure our model can be trained and evaluated correctly. The first of these is defining the filter that will be used to ensure that no *negative* statements generated by the corruption procedure are actually positives. This is simply done by concatenating our train and test sets. Now when negative triples are generated by the corruption strategy, we can check that they aren't actually true statements.  


In [17]:
positives_filter = subset

## Fitting the model

Once you run the next cell the model will train. 

On a modern laptop this should take ~3 minutes (although your mileage may vary, especially if you've changed any of the hyper-parameters above).

In [18]:
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train, early_stopping = False)

  0%|                                               | 0/200 [00:02<?, ?epoch/s]


InvalidArgumentError: ValueError: `generator` yielded an element of shape (3,) where an element of shape (?, 3) was expected.
Traceback (most recent call last):

  File "c:\users\johan\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\ops\script_ops.py", line 235, in __call__
    ret = func(*args)

  File "c:\users\johan\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\data\ops\dataset_ops.py", line 630, in generator_py_func
    "of shape %s was expected." % (ret_array.shape, expected_shape))

ValueError: `generator` yielded an element of shape (3,) where an element of shape (?, 3) was expected.


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]]

---
# 5.  Saving and restoring a model

Before we go any further, let's save the best model found so that we can restore it in future.

In [ ]:
from ampligraph.latent_features import save_model

In [ ]:
save_model(model, './airports_subset.pkl')

This will save the model in the ampligraph_tutorial directory as `best_model.pkl`.

.. we can then delete the model .. 

In [ ]:
#del model

.. and then restore it from disk! Ta-da! 

In [ ]:
#model = restore_model('./connectedTo_subset.pkl')

And let's just double check that the model we restored has been fit:

In [ ]:
if model.is_fitted:
    print('The model is fit!')
else:
    print('The model is not fit! Did you skip a step?')

## Running evaluation

In [ ]:
ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)


The ***ranks*** returned by the evaluate_performance function indicate the rank at which the test set triple was found when performing link prediction using the model. 

For example, given the triple:

    <House Stark of Winterfell, IN_REGION The North>
    
The model returns a rank of 7. This tells us that while it's not the highest likelihood true statement (which would be given a rank 1), it's pretty likely.


## Metrics

Let's compute some evaluate metrics and print them out.

We're going to use the mrr_score (mean reciprocal rank) and hits_at_n_score functions. 

- ***mrr_score***:  The function computes the mean of the reciprocal of elements of a vector of rankings ranks.
- ***hits_at_n_score***: The function computes how many elements of a vector of rankings ranks make it to the top n positions.


In [ ]:
mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

Now, how do we interpret those numbers? 

[Hits@N](http://docs.ampligraph.org/en/1.0.3/generated/ampligraph.evaluation.hits_at_n_score.html#ampligraph.evaluation.hits_at_n_score) indicates how many times in average a true triple was ranked in the top-N. Therefore, on average, we guessed the correct subject or object 53% of the time when considering the top-3 better ranked triples. The choice of which N makes more sense depends on the application.

The [Mean Reciprocal Rank (MRR)](http://docs.ampligraph.org/en/latest/generated/ampligraph.evaluation.mrr_score.html) is another popular metrics to assess the predictive power of a model.